In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
# import jax
# jax.config.update('jax_platform_name', 'cpu')
import numpyro
numpyro.set_host_device_count(8)
import os
import warnings

import sys
from pyprojroot import here
sys.path.append("../..") 
# from laos_gggi.statistics import get_distance_to_rivers

import pandas as pd
import numpy as np
import pymc as pm
import pytensor
import pytensor.tensor as pt
import matplotlib.pyplot as plt
import arviz as az
import geopandas as gpd
import xarray as xr

# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler, PolynomialFeatures

from laos_gggi import load_all_data, load_gpcc_data, load_emdat_data, load_shapefile, load_wb_data
from laos_gggi.data_functions.rivers_data_loader import load_rivers_data
# from laos_gggi.statistics import get_distance_to_rivers
from laos_gggi.data_functions.rivers_damage import create_hydro_rivers_damage, create_floods_rivers_damage
from laos_gggi.replication_data import create_replication_data
from laos_gggi.const_vars import COUNTRIES_ISO, ISO_DICTIONARY, LAOS_LOCATION_DICTIONARY
from laos_gggi.plotting import( configure_plot_style, plot_ppc_loopit, generate_plot_inputs, 
plotting_function, create_grid_from_shape, )

from laos_gggi.sample import sample_or_load
from laos_gggi.model import add_country_effect, get_distance_to, compute_center

from laos_gggi.data_functions.combine_data import load_model_df


ImportError: cannot import name 'load_model_df' from 'laos_gggi.data_functions.combine_data' (C:\Users\camil\Documents\Servicios\OCDE-GGI\laos-climate-change\notebooks\geo_damage_model\../..\laos_gggi\data_functions\combine_data.py)

In [17]:
# Set random seed
SEED = sum(list(map(ord, 'climate_bayes')))
rng = np.random.default_rng(SEED)

In [18]:
# Configure plot style
configure_plot_style()

# Importing and cleaning data

In [22]:
rep_df = create_replication_data()
emdat = load_emdat_data()
wb = load_wb_data()

#Load shapefiles
world = load_shapefile('world', repair_ISO_codes=True)
laos = load_shapefile('laos')
coastline = load_shapefile('coastline')
rivers = load_rivers_data()

In [23]:
# Import shapefiles
world = load_shapefile('world')
laos = load_shapefile('laos')

In [26]:
# Create the damagedf

damage_df = create_hydro_rivers_damage()

emdat_iso = damage_df.ISO.unique()
world_iso = world.ISO_A3.unique()
wb_iso = wb.index.get_level_values(0).unique()

# Codes in shapefile but not in EMDAT
common_codes = set(world_iso).intersection(set(emdat_iso)).intersection(set(wb_iso))
damage_df = damage_df.query('ISO in @common_codes').copy()
world = world.query('ISO_A3 in @common_codes').copy()
wb = (wb.reset_index().query('country_code in @common_codes').assign(year = lambda x: pd.to_datetime(x.year, format='%Y')).copy())

damage_df = (pd.merge(damage_df, wb, left_on=['ISO', 'year'], right_on=['country_code', 'year'], how='left')
             .dropna(subset=['population_density', 'gdp_per_cap', 'Population']))


  0%|          | 0/854 [00:00<?, ?it/s]

C:\Users\camil\Documents\Servicios\OCDE-GGI\laos-climate-change\notebooks\geo_damage_model\../..\laos_gggi\data_functions\rivers_damage.py:75: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  damage_df.to_file(os.path.join(data_path, RIVERS_HYDRO_DAMAGE_FILENAME))
C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'River Basin' to 'River Basi'
  ogr_write(
C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Total_Damage_Hydro' to 'Total_Dama'
  ogr_write(
C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Total_Affected_Hydro' to 'Total_Affe'
  ogr_write(
C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Field year create 

In [28]:
#Floods damages and people_affected
damage_df_f = create_floods_rivers_damage()
damage_df_f = pd.merge(damage_df_f, rep_df, right_on=["ISO", "year"], left_on= ["ISO", "year"], how = "left")

#Create different "ORDFLOW" dummies
damage_df_f["ORD_FLOW_1"] = 0
for i in damage_df_f.index.values:
    if damage_df_f.loc[i, "ORD_FLOW"] == 1:
        damage_df_f.loc[i, "ORD_FLOW_1"] = 1

#Create the variable intrecation between closest river and ORD_FLOW
damage_df_f["closest_river_int_ORD_FLOW_1"] = damage_df_f["closest_river"] * damage_df_f["ORD_FLOW_1"]

  0%|          | 0/766 [00:00<?, ?it/s]

C:\Users\camil\Documents\Servicios\OCDE-GGI\laos-climate-change\notebooks\geo_damage_model\../..\laos_gggi\data_functions\rivers_damage.py:163: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  damage_df_f.to_file(os.path.join(data_path, RIVERS_FLOODS_DAMAGE_FILENAME))
C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'River Basin' to 'River Basi'
  ogr_write(
C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Total_Damage_Flood' to 'Total_Dama'
  ogr_write(
C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Total_Affected_Flood' to 'Total_Affe'
  ogr_write(
C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\pyogrio\raw.py:709: RuntimeWarning: Field year cre

In [29]:
# Loading data
model_df = load_model_df()

#Delimiting data set
model_list_2 = ['ISO', 'Start_Year', "is_disaster", 'log_distance_to_river', 'log_distance_to_coastline',
                'is_island', "lat", "long" ]


distance_features_2 = ['log_distance_to_river', 'log_distance_to_coastline', 'is_island']

time_varying_features_2 = ["Population", "co2", "precip_deviation", "dev_ocean_temp", 'log_population_density', 'log_population_density_squared',
             'log_gdp_per_cap', 'log_gdp_per_cap_squared']

model_df_2 = model_df[model_list_2 + time_varying_features_2].dropna()

[autoreload of laos_gggi.data_functions.combine_data failed: Traceback (most recent call last):
  File "C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\importlib\__init__.py", line 131, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 995, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "C:\Users\camil\Documents\Servicios\OCDE-GGI\laos-climate-change\notebooks\geo_damage_model\../..\laos_gggi\data_functions\combine_data.py", line 11, in <module>
    from lao

NameError: name 'load_model_df' is not defined